In [9]:
# Solar System - USD Timesample Animation 
# Homework 2 - USD Studierstube
#
from pxr import Usd, UsdGeom, Gf, UsdShade, Sdf
import math
# Constants
OUT_FILENAME = "_assets/Animated_Solarsystem.usda"
FPS = 24
ANIM_LENGTH = 1000
UP_AXIS = "Y"
# Physical constants (km)
AU_KM = 149597870.7
EARTH_RADIUS_KM = 6371.0
distance_scale = 1.0 / AU_KM
earth_radius_visual = 0.05
radius_scale = earth_radius_visual / EARTH_RADIUS_KM
# Planet data: name, distance from Sun (km), radius (km), orbit-speed, spin-speed, color (r,g,b)
planets = [
    ("Mercury", 57909227, 2439.7, 1.5, 2.0, (0.5, 0.5, 0.5)),
    ("Venus",   108209475, 6051.8, 1.2, 1.0, (0.9, 0.7, 0.2)),
    ("Earth",   149597870.7, 6371.0, 1.0, 3.0, (0.2, 0.4, 1.0)),
    ("Mars",    227943824, 3389.5, 0.8, 2.5, (1.0, 0.4, 0.2)),
    ("Jupiter", 778340821, 69911, 0.6, 5.0, (0.9, 0.7, 0.5)),
    ("Saturn",  1426666422, 58232, 0.5, 4.0, (0.95, 0.85, 0.6)),
    ("Uranus",  2870658186, 25362, 0.4, 3.5, (0.5, 0.8, 0.9)),
    ("Neptune", 4498396441, 24622, 0.35, 3.0, (0.2, 0.4, 0.8)),
]
def create_stage(filename):
    stage = Usd.Stage.CreateNew(filename)
    # stage = Usd.Stage.CreateInMemory() 
    # Set up default prim
    stage.SetDefaultPrim(stage.DefinePrim("/SolarSystem", "Xform"))
    return stage
def create_preview_material(stage, name, color_rgb):
    # UsdShade PreviewSurface material with diffuse color.
    mat_path = Sdf.Path(f"/Materials/{name}Mat")
    mat_prim = stage.DefinePrim(mat_path, "Material")
    material = UsdShade.Material(mat_prim)
    shader_path = mat_path.AppendChild("PreviewSurfaceShader")
    shader_prim = stage.DefinePrim(shader_path, "Shader")
    shader = UsdShade.Shader(shader_prim)
    shader.CreateIdAttr("UsdPreviewSurface")
    # Set diffuse color
    shader.CreateInput("diffuseColor", Sdf.ValueTypeNames.Color3f).Set(Gf.Vec3f(*color_rgb))
    shader.CreateInput("roughness", Sdf.ValueTypeNames.Float).Set(0.6)
    material.CreateSurfaceOutput().ConnectToSource(shader.ConnectableAPI(), "surface")
    return material
def bind_material_to_prim(material, prim):
    UsdShade.MaterialBindingAPI(prim).Bind(material)
def create_planet(stage, parent_path, name, distance_km, radius_km, color_rgb, angle_deg=0.0):
    # Create a sphere representing a planet. Place at angle_deg around the sun in the XZ plane.
    planet_path = parent_path.AppendChild(name)
    sphere_prim = stage.DefinePrim(planet_path, "Sphere")
    sphere = UsdGeom.Sphere(sphere_prim)
    # scaled radius and position
    radius = radius_km * radius_scale
    sphere.GetRadiusAttr().Set(radius)
    # position in the XZ plane (Y up)
    distance = distance_km * distance_scale
    angle = math.radians(angle_deg)
    x = math.cos(angle) * distance
    z = math.sin(angle) * distance
    # Set translate
    UsdGeom.XformCommonAPI(sphere_prim).SetTranslate(Gf.Vec3d(x, 0.0, z))
    # Create and bind material
    mat = create_preview_material(stage, name, color_rgb)
    bind_material_to_prim(mat, sphere_prim)
    #
    return sphere_prim
def bind_material_to_prim(material, prim):
    UsdShade.MaterialBindingAPI(prim).Bind(material)
def create_orbit_curve(stage, parent_path, name, radius_km, segments=180):
    # Create UsdGeom.Curve for orbit curve in the XZ plane
    curve_path = parent_path.AppendChild(f"{name}_Orbit")
    curve_prim = stage.DefinePrim(curve_path, "BasisCurves")
    curve = UsdGeom.BasisCurves(curve_prim)
    # points making up the circle
    pts = []
    radius = radius_km * distance_scale
    for i in range(segments+1):
        theta = 2.0 * math.pi * (i / segments)
        x = math.cos(theta) * radius
        z = math.sin(theta) * radius
        pts.append(Gf.Vec3f(x, 0.0, z))
    curve.CreatePointsAttr().Set(pts)
    curve.CreateCurveVertexCountsAttr().Set([len(pts)])
    # make curve renderable as a ribbon/line
    curve.CreateWidthsAttr().Set([0.01])
    # color
    curve.CreateDisplayColorAttr().Set([Gf.Vec3f(0.6, 0.6, 0.6)])
    return curve_prim
def add_orbit_animation(xform, start_frame, end_frame, degrees_per_frame, distance):
   # planet_xform_api = UsdGeom.XformCommonAPI(xform)
    translate_attr = xform.GetAttribute("xformOp:translate")
    # Orbit-Eigenschaften
    distance = distance * distance_scale
    for frame in range(ANIM_LENGTH):
        angle_deg = ((frame * degrees_per_frame) / ANIM_LENGTH) * 360.0
        angle_rad = math.radians(angle_deg)
        x = distance * math.cos(angle_rad)
        z = distance * math.sin(angle_rad)
        translate_attr.Set(Gf.Vec3d(x, 0, z), time=frame)
def add_spin_animation(xform, start_frame, end_frame, spin_speed):
    # Planet Spin Animation 
    xform_api = UsdGeom.XformCommonAPI(xform)
    for frame in range(ANIM_LENGTH):
        angle = float((frame * spin_speed) % 360.0)
        xform_api.SetRotate((0.0, angle, 0.0), time=frame)        
#########################################
stage = create_stage(OUT_FILENAME)
root_path = Sdf.Path("/SolarSystem")
# Create Sun at origin
sun_path = root_path.AppendChild("Sun")
sun_prim = stage.DefinePrim(sun_path, "Sphere")
sun = UsdGeom.Sphere(sun_prim)
sun_radius = 0.5  # units (half unit)
sun.GetRadiusAttr().Set(sun_radius)
# Sun Material
sun_mat = create_preview_material(stage, "Sun", (1.0, 0.9, 0.2))
bind_material_to_prim(sun_mat, sun_prim)
#
stage.SetFramesPerSecond(FPS)
stage.SetStartTimeCode(0)
stage.SetEndTimeCode(ANIM_LENGTH)
UsdGeom.SetStageUpAxis(stage, UP_AXIS)
world = UsdGeom.Xform.Define(stage, "/World")
solar = UsdGeom.Xform.Define(stage, "/World/SolarSystem")
stage.DefinePrim("/Materials", "Scope")
# Create planets (and orbits) — place each at a slightly different angle so they don't overlap in a line
angle_step = 360.0 / len(planets)
for i, (name, dist_km, radius_km, orbitspeed, spinspeed, color) in enumerate(planets):
    angle = i * angle_step + 10.0  # offset a bit
    planet = create_planet(stage, root_path, name, dist_km, radius_km, color, angle_deg=angle)
    # Create orbit visualization for planets (skip Moon to avoid clutter maybe)
    if name != "Moon":
        orbit_xf = create_orbit_curve(stage, root_path, name, dist_km, segments=180)
    # Animations
    if name != "Sun":
      add_orbit_animation(planet, 0, ANIM_LENGTH, orbitspeed * 5.0, dist_km) 
      add_spin_animation(planet, 0, ANIM_LENGTH, 2.0 * 10.0)

# Save stage
stage.GetRootLayer().Save()
print(f"Saved stage: {OUT_FILENAME}")

Saved stage: _assets/Animated_Solarsystem.usda
